# Simulations with iUma22


In [1]:
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from cobra import Model #, Reaction, Metabolite
from cobra.io import read_sbml_model
from escher import Builder

In [2]:
ModelFile = join('..','model','iUma22.xml')
model=read_sbml_model(ModelFile)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,2.2,6,100.00%
h2o_e,EX_h2o_e,3.8,0,0.00%
nh4_e,EX_nh4_e,1.113,0,0.00%
o2_e,EX_o2_e,0.01102,0,0.00%
so4_e,EX_so4_e,0.01387,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
adn_e,EX_adn_e,-2.391,10,80.04%
fum_e,EX_fum_e,-0.02976,4,0.40%
gly_e,EX_gly_e,-0.3169,2,2.12%
mal__L_e,EX_mal__L_e,-1.303,4,17.44%


In [5]:
model.metabolites[0].annotation

{'sbo': 'SBO:0000247',
 'biocyc': 'META:DIHYDROPTERIN-CH2OH-PP',
 'chebi': '57602',
 'inchi': 'InChI=1S/C7H11N5O8P2/c8-7-11-5-4(6(13)12-7)10-3(1-9-5)2-19-22(17,18)20-21(14,15)16/h1-2H2,(H,17,18)(H2,14,15,16)(H4,8,9,11,12,13)/p-3',
 'kegg.compound': 'C04807',
 'pubchem.compound': '25244074'}

## Cofeeding of Glc+Ace

In [ ]:
model.add_boundary(metabolite=model.metabolites.h_e, type='exchange', reaction_id='EX_h_e', lb=0, ub=1000)
medium=model.medium
with model:
    medium['EX_glc__D_e']= 5#7.69
#     medium['EX_ac_e']=10
    model.medium=medium
    solution=model.optimize()
    print(solution.fluxes['BIOMASS_REACTION'])
    print(model.summary())

In [ ]:
NSamp = 6
Glucose_list = np.linspace(.1,6,NSamp) # # np.array([5,7,9,11,13,15])
Acetate_list = np.linspace(0,2,NSamp)
# medium_list = np.array([[x,y] for x in Glucose_list for y in Acetate_list])
Glc_list, Ace_list = np.meshgrid(Glucose_list, Acetate_list)
medium_list = np.transpose(np.reshape(np.array([Glc_list, Ace_list]),(2,-1)))
# print(medium_list)

In [ ]:
medium=model.medium
Bio_list = list()
for z in medium_list:
    x=z[0]
    y=z[1]
    with model:
        medium['EX_glc__D_e']=x
        medium['EX_ac_e']=y
        model.medium=medium
        solution=model.optimize()
        Bio_list.append(solution.fluxes['BIOMASS_REACTION'])
        
Bio_list = np.reshape(np.array(Bio_list),(-1,1))
Z = np.reshape(Bio_list,(NSamp,NSamp))
plt.contourf(Glc_list, Ace_list, Z, 20, cmap='coolwarm')
plt.colorbar(label='growth /h')
plt.xlabel('Glc in mmol/gCDW/h')
plt.ylabel('Ace in mmol/gCDW/h')
# plt.savefig('Umay-Glc-Ace_default.png')

### forcing proton export

In [ ]:
Bio_list = list()
for z in medium_list:
    x=z[0]
    y=z[1]
    with model:
        medium['EX_glc__D_e']=x
        medium['EX_ac_e']=y
        model.medium=medium
        model.reactions.EX_h_e.lower_bound = np.power(2.33,y)
        solution=model.optimize()
        Bio_list.append(solution.fluxes['BIOMASS_REACTION'])
        
Bio_list = np.reshape(np.array(Bio_list),(-1,1))
Z = np.reshape(Bio_list,(NSamp,NSamp))
plt.contourf(Glc_list, Ace_list, Z, 20, cmap='coolwarm')
plt.colorbar(label='growth /h')
plt.xlabel('Glc in mmol/gCDW/h')
plt.ylabel('Ace in mmol/gCDW/h')
# plt.savefig('Umay-Glc-Ace_InhHigh.png')

## Yield simulations of itaconate from different substrates

Simulations are performed to identify the maximum yield for itaconate from various substrates.

In [ ]:
Substrate_List = ['EX_glc__D_e','EX_ac_e']
Sol_Dict = dict.fromkeys(Substrate_List)
Yield_Dict = dict.fromkeys(Substrate_List)
for Substrate in Substrate_List:
    with model:
        medium=model.medium
        medium['EX_glc__D_e']=0
        medium['EX_co2_e']=0
        medium[Substrate]=10
#         medium['EX_for_e'] = 1
        model.medium=medium
        model.objective = 'EX_itacon_e'
        solution=model.optimize()
#         print(model.summary())
        Sol_Dict[Substrate] = solution
        Sub_Name = model.reactions.get_by_id(Substrate).reaction.split()[0]
        try:
            Sub_Catom = int(model.metabolites.get_by_id(Sub_Name).formula[1])
        except ValueError:
            Sub_Catom = 1
        Yield_Dict[Substrate] = round(solution.objective_value,2)*5/(Sub_Catom*medium[Substrate])

print('Yield: ')
Yield_Dict

In [31]:
# cofeed itaconate yield
with model:
    medium=model.medium
    medium['EX_glc__D_e']=10
    medium['EX_ac_e']=0
    medium['EX_co2_e']=10
    model.medium=medium
    model.objective = 'EX_itacon_e'
    cofeed_sol = model.optimize()
    cofeed_yield = round(cofeed_sol.objective_value,2)*5/(6*medium['EX_glc__D_e'] + 2*medium['EX_ac_e'])

print('Yield: ')
cofeed_yield

Yield: 


1.1108333333333333

In [32]:
# load U. maydis ESCHER maps
builder=Builder()

Escher_Central = join('Maps','iUma22_MetMap_TCA.json')
Escher_Glycine = join('Maps','iUma22_MetMap_glycine.json')
builder = Builder(
    map_json=Escher_Central,
    model = model, # 'iCL1079.json',
)
# Run FBA with the model and add the flux data to the map
solution = builder.model.optimize()
builder.reaction_data = cofeed_sol.fluxes# Sol_Dict['EX_glc__D_e'].fluxes

builder.save_html('../code/example_map.html')

In [26]:
[print(reaction.name) for reaction in model.reactions for genes in reaction.genes if 'UMAG_02376' in genes.name];


PRTRANS-RXN
ANTHRANSYN-RXN
PRAISOM-RXN
IGPSYN-RXN
